In [1]:
# Connecting to Azure ML Workspace
from azureml.core import Workspace, Model

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep=' | ')

# Loading the model from the Azure ML registry
model = Model(workspace=ws, name="pimadiabetes_logistic_regression_model")
print("Model loaded:", model.name)



edohkodji-machinelearn | defi | eastus2
Model loaded: pimadiabetes_logistic_regression_model


In [2]:
# Definition of the inference environment
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_pip_requirements(name="pima-env", file_path="requirements.txt")

inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)


In [7]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name="pima-diabetes-deploy-02",  
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)


/tmp/ipykernel_3155/4211749718.py:5: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-05-19 14:02:49+00:00 Creating Container Registry if not exists.
2025-05-19 14:02:50+00:00 Use the existing image.
2025-05-19 14:02:50+00:00 Generating deployment configuration.
2025-05-19 14:02:55+00:00 Submitting deployment to compute..
2025-05-19 14:03:04+00:00 Checking the status of deployment pima-diabetes-deploy-02..
2025-05-19 14:04:33+00:00 Checking the status of inference endpoint pima-diabetes-deploy-02.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [9]:
# Testing the API with a prediction request
import requests
import json

sample_data = json.dumps({"data": [[8, 183, 64, 29.15, 155.54, 23.3, 0.672, 32]]})
headers = {"Content-Type": "application/json"}

response = requests.post(service.scoring_uri, data=sample_data, headers=headers)
print("Réponse:", response.json())

Réponse: [1]
